In [3]:
import pandas as pd
import os
import re

In [4]:
def load_files (path_file):
    story = open(path_file, encoding = 'utf-8')
    file_text = story.read()
    story.close()
    return file_text

In [5]:
def article_summary (file_text):
    
    file_text = file_text.replace('\n',' ').replace("\'","")
    start_summary = file_text.find('@highlight') #summaries are identified by '@highlight'
    article = file_text[:start_summary]
    summary = file_text[start_summary+len('@highlight'):]

    summaries_list = summary.split('@highlight')
    summaries_list = [summ.strip() for summ in summaries_list]

    summaries_text = summary.replace('@highlight', '. ').strip()

    return article, summaries_text, summaries_list

In [6]:
def callback( str ):
    return str.replace('.', '')

In [46]:
def load_data (stories_directory):
    articles = []
    summary_text = []
    summary_list = []
    
    i=0
    for filename in os.listdir(stories_directory)[:20000]:
        #print(filename)
        
        path = stories_directory + '/'+ filename
        file_text = load_files(path)
        article, summaries_text, summaries_list = article_summary(file_text)

        if len(article) != 0:
            print(i)
            article = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', ' ', str(article)) #remove website like 'http://website.com'
            article = re.sub(r'www[\s]?[.][\s]?(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', ' ', str(article)) #remove website like 'www.website.com'
            article = re.sub(r'[\(|\[]?[A-Za-z]+\.com[\s|\)|\]]', ' ', str(article)) #remove website like 'website.com'
            article = re.sub(r'[0-9]+\s?[.]\s?[0-9]+', 'xxxx', article) #remove decimal numbers
            article = re.sub(r'\b([a-zA-Z]\.)([a-zA-Z]\.)+', lambda m: callback(m.group()), article) #acronimi
            article = article.replace('.', '. ')
            articles.append(article)
            summary_text.append(summaries_text)
            summary_list.append(summaries_list)



        i+=1

    data = pd.DataFrame()
    data['article'] = articles
    data['summary_string'] = summary_text
    data['summary_list'] = summary_list

    return data


In [ ]:
data = load_data('D:/UNIVERSITA_2/Magistrale/TextMining/PROGETTO_text_mining/cnn_stories/cnn/stories')
data.to_csv('article_summaries_noNaN_prep_20000_2.csv')